# Pytorch tutorial

## What is pytorch?
- Numpy를 대체하며 GPU연산을 지원하는 tensor 구조 지원  
CUDA(NVIDIA에서 gpu를 통한 연산을 위해 만든 API)와 cuDNN(CUDA를 활용하여 딥러닝 gpu연산을 지원하는 API)을 활용하여 연산 가속  
일반적으로 CUDA와 cuDNN을 활용하는 연산은 CPU연산의 15배정도 가속을 가져옴. 


- backward() 함수를 통한 그래프 미분 연산 지원

## Pytorch vs Tensorflow

#### 둘 모두 GPU를 활용하는 framework

2022년 최신버전 기준으로 Tensorflow와 Pytorch의 차이점이 거의 없어졌다.
Tensorflow가 2020년 이후로 Pytorch 스럽게 많이 바꿨다
Pytorch도 tensorflow스럽게 바뀐 부분도 있다 Keras를 따라하려고 노력
Keras는 tensorflor를 쉽게 쓰도록 돕는 라이브러리

딥러닝이 이루어지기 위한 3가지 조건
1. 많은 데이터
2. 좋은 성능의 GPU
3. 미분연산
Tensorflow나 Pytorch는 **미분연산**을 하기 위해 연산을 Graph화 시킨다
연산자와 기호들이 Node화 됐다

- Tensorflow

기본적으로 C++기반, Python의 개발 철학과 맞지 않는 부분이 많다
Python인데 C++스럽게 코딩해야 하는 부분이 있다

`Define and Run`방식  
연산 그래프를 미리 만들어두고, 실제 연산시 값을 전달하여 결과를 얻음(정적)  
직관적이지 않고 그래프를 정의하는 부분과 실행하는 부분이 분리되어 코드가 길어짐  
최적화에서 장점

동적, 정적 큰 차이가 없어 보이는데..
if 연산을 사칙연산으로 구현하기 어려워
Tensorflow는 정적으로 만들어두기 때문에 cond라는 메소드로 처리를 하는데, 버그도 많고 다루기가 어려워
Tensorflow 2.0이 나오면서 동적 그래프를 지원하기 시작했어요.

Pytorch는 if 연산 하기 쉬워, 값이 들어올 때 그래프 만들기 때문에, 값 들어오면서 if연산을 해

- Pytorch(Facebook)

Tensorflow를 개선하기 위해 나온 라이브러리
Python스럽게 코딩할 수 있도록 돕는다

`Define by Run` 방식  
연산 그래프를 미리 만들어두지 않고 값이 할당되어 전달되는 과정에서 그래프가 작성(동적)  
직관적이고 간단한 코드  
(Pytorch측의 주장으로는)Tensorflow보다 평균 2.5배정도 빠른 속도(적어도 밀리지는 않음)
속도상의 차이는 거의 없다.

## Tensor

Tensor: Numpy의 ndarray와 유사한 matrix자료구조. GPU연산 가속이 가능

In [1]:
import torch
import numpy as np

### Tensor 생성

In [2]:
not_initialized = torch.empty(3,4)
not_initialized

tensor([[1.6471e-35, 0.0000e+00, 3.3631e-44, 0.0000e+00],
        [       nan, 0.0000e+00, 1.1578e+27, 1.1362e+30],
        [7.1547e+22, 4.5828e+30, 1.2121e+04, 7.1846e+22]])

In [3]:
random_initialized = torch.rand(3, 4)
random_initialized

tensor([[0.5991, 0.6654, 0.8204, 0.2308],
        [0.5472, 0.9527, 0.1538, 0.6657],
        [0.3416, 0.3326, 0.5274, 0.9126]])

In [4]:
zero_initialized = torch.zeros(3, 4)
zero_initialized

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [5]:
list_initialized = torch.tensor([[1,2],[3,4]])
list_initialized

tensor([[1, 2],
        [3, 4]])

In [6]:
arange_lnitiaized = torch.arange(10)
arange_lnitiaized

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Tensor와 numpy간의 변환

In [7]:
x = torch.ones(10)
x, type(x)

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), torch.Tensor)

In [8]:
y = x.numpy()
y, type(y)

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), numpy.ndarray)

In [9]:
z = torch.from_numpy(y)
z

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

#### 주의사항 - tensor와 ndarray는 메모리공간을 공유함

numpy buffer view 구조 tensor에서도 똑같이 사용
z, y는 buffer는 공유하고 view만 자기만의 view를 가지고 있는거야

In [10]:
x.add_(1)

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [11]:
x

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [12]:
y, z

(array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float32),
 tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]))

### Tensor 특성

shape dimension axis 똑같이 가지고 있어

In [13]:
list_initialized.shape, list_initialized.ndim

(torch.Size([2, 2]), 2)

In [14]:
list_initialized.size()

torch.Size([2, 2])

### Tensor reshape vs view

-  두 함수는 기본적으로 같은 기능을 하나, view는 shallow copy이다.
-  그러나, reshape이 deep copy인 것은 아니다. reshape은 상황에 따라 deep/shallow를 넘나든다.
-  Reshape은 Contiguous한 operate를 지원하나, view는 지원하지 않는다.

Tensor와 numpy에서 다른 딱 하나가 reshape
옛날에는 Tensor에 view밖에 없었어
reshape 썼으면 좋겠어, 라고 지속적으로 유저들이 문의해서 만들어줬어
numpy reshape와 기능이 약간 달라


In [15]:
list_initialized

tensor([[1, 2],
        [3, 4]])

In [16]:
list_initialized.t()

tensor([[1, 3],
        [2, 4]])

In [17]:
print(list_initialized.reshape(-1, 1))
print(list_initialized.view(-1, 1))

tensor([[1],
        [2],
        [3],
        [4]])
tensor([[1],
        [2],
        [3],
        [4]])


In [18]:
list_initialized.t().reshape(-1, 1)
# 행렬의 형태를 바꾸는 연산을 Contiguous라고 하는데, transpose같은 contiguous 연산에는 view가 작동하지 않아
# Contiguous연산이 적용 되면 buffer view 공간에 Contiguous 연산 됐다고 flag 써두는데 그걸 바탕으로 바꾸는게 이상해
# reshape는 최신꺼라서 그걸 해줘, 근데 view는 안해줘.. 왜 안해주는지는 안써있어 모르겠어..
# deep copy인지 shallow copy인지 신경 안써도 되도록 코딩하게 만드는 게 python style이다 라고 해서 그렇게 만들었대..
# contiguous 있으면 reshape 쓰고 아니면 view 쓰자. 그정도만 생각하자

tensor([[1],
        [3],
        [2],
        [4]])

In [19]:
list_initialized.t().view(-1, 1) # Error!

RuntimeError: ignored

### Tensor의 배열 연산

인덱싱, 슬라이싱, 선형대수 등 각종 연산 numpy와 동일하게 수행 가능

#### 기본연산

In [20]:
x = torch.arange(1,11).reshape(2, 5)
y = torch.arange(1, 20, step=2).reshape(2, 5)
x, y

(tensor([[ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10]]), tensor([[ 1,  3,  5,  7,  9],
         [11, 13, 15, 17, 19]]))

In [21]:
x+y, torch.add(x, y)

(tensor([[ 2,  5,  8, 11, 14],
         [17, 20, 23, 26, 29]]), tensor([[ 2,  5,  8, 11, 14],
         [17, 20, 23, 26, 29]]))

In [22]:
x-y, torch.sub(x, y)

(tensor([[ 0, -1, -2, -3, -4],
         [-5, -6, -7, -8, -9]]), tensor([[ 0, -1, -2, -3, -4],
         [-5, -6, -7, -8, -9]]))

In [24]:
x*y

tensor([[  1,   6,  15,  28,  45],
        [ 66,  91, 120, 153, 190]])

In [25]:
y%x

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

#### 행렬곱

In [26]:
x.matmul(y.reshape(5, 2))

tensor([[175, 205],
        [400, 480]])

#### 바꿔치기(in-place) 와 반환하기(out-of-place)

In [27]:
x

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [28]:
x.add(3)

tensor([[ 4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13]])

In [29]:
x

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [30]:
x.add_(3)

tensor([[ 4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13]])

In [31]:
x

tensor([[ 4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13]])

#### 헷갈리는 경우

In [32]:
z = x.numpy()
x = x.add(3)
print('x:',x)
print('z:',z)
print('========================')
x.add_(3)
print('x:', x)
print('z:', z)

x: tensor([[ 7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16]])
z: [[ 4  5  6  7  8]
 [ 9 10 11 12 13]]
x: tensor([[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
z: [[ 4  5  6  7  8]
 [ 9 10 11 12 13]]


#### Indexing and slicing

In [33]:
x[0, 3]

tensor(13)

In [34]:
x[:, 1:4]

tensor([[11, 12, 13],
        [16, 17, 18]])

### Aggregation

In [35]:
x

tensor([[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])

In [36]:
x.sum()

tensor(145)

In [37]:
x.sum(dim=0), x.sum(axis=0)

(tensor([25, 27, 29, 31, 33]), tensor([25, 27, 29, 31, 33]))

In [38]:
x.sum(dim=1), x.sum(axis=1)

(tensor([60, 85]), tensor([60, 85]))

#### Can't use mean() on integers

aggregation 할 때 주의해라

numpy([1,2,3,4,5]).mean() 2.5로 잘 나오는데


In [39]:
x.dtype, x

(torch.int64, tensor([[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]))

In [40]:
try:
    print(x.mean(dim=0))
except Exception as exc:
    print(exc)

    # input이 integer하고 output이 float이라서 작동을 안해 더 type에 strict해
    # pytorch에서는 gpu에서 정확하게 행렬연산을 해야하기 때문에 더 strict해
    # numpy에서는 해줘..

mean(): input dtype should be either floating point or complex dtypes. Got Long instead.


In [41]:
# x = torch.arange(1,11, dtype=float).reshape(2, 5)
x = x.float()
x

tensor([[10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.]])

In [42]:
x.mean(), x.mean(dim=0)

(tensor(14.5000), tensor([12.5000, 13.5000, 14.5000, 15.5000, 16.5000]))

### Broadcasting

In [43]:
x = torch.arange(1,11).reshape(1, -1)
y = torch.arange(1, 20, step=2).reshape(-1, 1)
print(x)
print(y)

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])
tensor([[ 1],
        [ 3],
        [ 5],
        [ 7],
        [ 9],
        [11],
        [13],
        [15],
        [17],
        [19]])


In [44]:
x+y

tensor([[ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
        [14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        [16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
        [18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

## AUTOGRAD: 자동미분

`pytorch` 신경망의 중심. `backward()`를 가능케 하는 패키지  
`Tensor`의 모든 연산에 대해 자동 미분을 제공

`pytorch.Tensor` class에는 `.required_grad` 속성이 존재  
`.required_grad`를 True로 세팅하면 해당 tensor를 기반으로 이루어신 모든 연산을 추적(track)하기 시작함  
모든 연산이 완료된 후, `backward()`를 호출하면 모든 gradient가 자동으로 게산(`Tensor.grad` 속성에 gradient가 누적됨)

Autograd의 다른 중요 class는 `Function`class  
`Tensor`와 `Function`class는 연결되어 있으며, 모든 연산을 encode하여 acyclic graph를 생성  
각각의 tensor는 `.grad_fn`속성을 가지며 이는 `Tensor`를 생성한 `Function`을 참조함

AUTOGRAD

pytorch의 package

backward()는 편미분을 해라 라고 명령 내리는 것

required grad, grad가 pytorch에서만 buffer view 공간에 추가적으로 붙어있는 값들

미분 계산은 복잡해 사용자가 생성한 모든 변수를 미분하려고 하면 계산량이 많아져

그래서 required grad True false를 통해서 필요한 변수에 대한 편미분 값만 가져올 수 있어



In [45]:
x = torch.ones(5, 2, requires_grad=True)
# x = torch.arange(10, dtype=torch.float32).reshape(5, 2)
# x.requires_grad = True
x

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)

In [46]:
y = x + 10

In [47]:
y

tensor([[11., 11.],
        [11., 11.],
        [11., 11.],
        [11., 11.],
        [11., 11.]], grad_fn=<AddBackward0>)

y는 연산의 결과로 생성된 tensor이므로 grad_fn을 가짐

In [48]:
y.grad_fn
# back propagation할 때 여기있는 친구가 Add를 하고 있는 것을 참고 해

In [49]:
z = y * y * 10

In [50]:
z

tensor([[1210., 1210.],
        [1210., 1210.],
        [1210., 1210.],
        [1210., 1210.],
        [1210., 1210.]], grad_fn=<MulBackward0>)

In [51]:
out = z.mean()

In [52]:
z, out

(tensor([[1210., 1210.],
         [1210., 1210.],
         [1210., 1210.],
         [1210., 1210.],
         [1210., 1210.]], grad_fn=<MulBackward0>),
 tensor(1210., grad_fn=<MeanBackward0>))

### 변화도(Gradient) 계산)

In [53]:
out.backward()
# required grad True인 아이들에 대해 편미분 값을 구해라

In [55]:
print(x.grad)
print(x)

tensor([[22., 22.],
        [22., 22.],
        [22., 22.],
        [22., 22.],
        [22., 22.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)


#### Jacobian matrix 활용

In [ ]:
# 행렬 미분을 Jacobian matrix라고 한다
# 이건 거의 활용하지 않아.

x = torch.ones(5, 2, requires_grad=True)
y = x + 10
z = y * y * 10
out = z.mean(axis=1)
out

In [ ]:
out.backward(torch.tensor([1., 3.,1.,1.,1.]))

In [ ]:
print(x.grad)

## DataLoader

torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,  num_workers=0,...)

데이터를 학습모델에 통째로 넣지 않고 batch 단위로 나누어 학습시킬 때 활용.  
모든 데이터를 나누고 나눈 데이터를 차례로 넣어주는 과정을 대신해줌
- 용어  
 - epoch: 모든 training data 혹은 모든 training data를 학습시킨 상태
 - batch: batch_size만큼의 training data만 학습시키기 위한 소분 데이터


In [ ]:
x = [[73, 80, 75],
   [93, 88, 93],
   [89, 91, 90],
   [96, 98, 100],
   [73, 66, 70]]
x = torch.FloatTensor(x)
x

In [ ]:
data_loader = torch.utils.data.DataLoader(x,
                            batch_size=2, 
                            shuffle=True, 
                            num_workers=0)

In [ ]:
epochs = 2
for epoch in range(epochs):
    for step, batch in enumerate(data_loader):
        print(step, batch)

## CUDA Tensor

### GPU로 이동

In [ ]:
# 뭘 GPU에 올려야 하냐?
# 데이터, 모델, loss function 3가지 올리면 돼

if torch.cuda.is_available():
    x = x.to(torch.device('cuda'))
    print(type(x))
    print(x.device)

#### 생성과 함께 gpu에 할당

In [ ]:
if torch.cuda.is_available():
    k = torch.full((2, 3), 1, device=torch.device('cuda'))
    print(type(k))
    print(k.device)
    # 여러개의 GPU가 있다면 0번째 GPU에 올라갔다고 이야기 하는 것

### CPU로 이동

In [ ]:
if torch.cuda.is_available():
    x = x.to(torch.device('cpu'))
    print(type(x))
    print(x.device)